# Profiling main skforecast classes and methods

In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'c:\\Users\\jaesc2\\GitHub\\skforecast'

In [2]:
import platform
import psutil
import skforecast
import pandas as pd
import numpy as np
import scipy
import sklearn
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.model_selection import grid_search_forecaster_multiseries
from skforecast.utils.utils import align_series_and_exog_multiseries
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer
from skforecast.preprocessing import series_long_to_dict
from skforecast.preprocessing import exog_long_to_dict
from skforecast.datasets import fetch_dataset

%load_ext pyinstrument
%load_ext line_profiler

In [3]:
n = 5000
n_series = 500
series = pd.DataFrame(
    {f"series_{i}": np.random.randint(1, 100, n) for i in range(1, n_series)
    },
    index=pd.date_range(start="2023-01-01", periods=n, freq="D"),
)

freq = series.index.freq
series.index.name = "datetime"
series = series.reset_index()
series = pd.melt(series, id_vars="datetime", var_name="series_id", value_name="value")
series = series.groupby("series_id", sort=False).apply(
    lambda x: x.set_index("datetime").asfreq("D"), include_groups=False
)
display(series)

exog = pd.DataFrame(
    {
        "exog_1": np.random.randint(1, 10, n),
        "exog_2": np.random.randint(1, 10, n),
        "exog_3": np.random.randint(1, 10, n),
        "exog_4": np.random.randint(1, 10, n),
        "exog_5": np.random.randint(1, 10, n),
        "exog_6": np.random.randint(1, 10, n),
    },
    index=pd.date_range(start="2023-01-01", periods=n, freq="D"),
)
exog.index.name = "datetime"
exog = [exog.assign(series_id=f"series_{i}") for i in range(1, n_series)]
# exog = [exog_i.copy().sample(frac=0.8).sort_index() for exog_i in exog]
exog = pd.concat(exog)
exog = exog.set_index(["series_id", exog.index])
display(exog)

# Formato dict
series_dict = {
    sid: series.loc[sid]['value'] for sid in series.index.levels[0]
}

exog_dict = {
    sid: exog.loc[sid]for sid in exog.index.levels[0]
}

value
series_id  datetime         
series_1   2023-01-01     50
           2023-01-02     25
           2023-01-03     74
           2023-01-04     31
           2023-01-05     28
...                      ...
series_499 2036-09-04     67
           2036-09-05     75
           2036-09-06     31
           2036-09-07     11
           2036-09-08     46

[2495000 rows x 1 columns]

exog_1  exog_2  exog_3  exog_4  exog_5  exog_6
series_id  datetime                                                  
series_1   2023-01-01       2       9       3       2       9       6
           2023-01-02       7       4       2       7       5       2
           2023-01-03       5       4       7       3       4       7
           2023-01-04       7       8       4       2       3       7
           2023-01-05       6       9       7       3       1       5
...                       ...     ...     ...     ...     ...     ...
series_499 2036-09-04       5       2       6       5       9       7
           2036-09-05       7       3       7       2       3       2
           2036-09-06       5       7       9       3       9       5
           2036-09-07       1       1       7       9       4       4
           2036-09-08       7       3       9       3       9       4

[2495000 rows x 6 columns]

In [43]:
type(series.loc['series_1'])

pandas.core.frame.DataFrame

In [41]:
na = series.columns[0]
series[na]

series_id   datetime  
series_1    2023-01-01    50
            2023-01-02    25
            2023-01-03    74
            2023-01-04    31
            2023-01-05    28
                          ..
series_499  2036-09-04    67
            2036-09-05    75
            2036-09-06    31
            2036-09-07    11
            2036-09-08    46
Name: value, Length: 2495000, dtype: int32

In [37]:
%%timeit

series.iloc[:, 0]

30.8 μs ± 647 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
series_dict

{'series_1': datetime
 2023-01-01    50
 2023-01-02    25
 2023-01-03    74
 2023-01-04    31
 2023-01-05    28
               ..
 2036-09-04    67
 2036-09-05    86
 2036-09-06    94
 2036-09-07    96
 2036-09-08    95
 Freq: D, Name: value, Length: 5000, dtype: int32,
 'series_2': datetime
 2023-01-01    38
 2023-01-02    91
 2023-01-03    58
 2023-01-04     6
 2023-01-05     4
               ..
 2036-09-04    39
 2036-09-05    23
 2036-09-06    21
 2036-09-07    51
 2036-09-08    49
 Freq: D, Name: value, Length: 5000, dtype: int32,
 'series_3': datetime
 2023-01-01    56
 2023-01-02    20
 2023-01-03    20
 2023-01-04    29
 2023-01-05    85
               ..
 2036-09-04    93
 2036-09-05    19
 2036-09-06    74
 2036-09-07    71
 2036-09-08    70
 Freq: D, Name: value, Length: 5000, dtype: int32,
 'series_4': datetime
 2023-01-01    99
 2023-01-02    44
 2023-01-03    95
 2023-01-04    51
 2023-01-05    84
               ..
 2036-09-04    94
 2036-09-05    52
 2036-09-06    52
 20

In [4]:
series_no_idx = series.reset_index(["series_id", "datetime"])
series_no_idx

,series_id,datetime,value
0,series_1,2023-01-01,50
1,series_1,2023-01-02,25
2,series_1,2023-01-03,74
3,series_1,2023-01-04,31
4,series_1,2023-01-05,28
...,...,...,...
2494995,series_499,2036-09-04,67
2494996,series_499,2036-09-05,75
2494997,series_499,2036-09-06,31
2494998,series_499,2036-09-07,11


In [7]:
%%timeit

series_no_idx.groupby('series_id', observed=True).size()
series_no_idx.groupby('series_id', observed=True).size()

321 ms ± 6.36 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit

data_grouped = series.groupby('series_id', observed=True)
data_grouped.size()
data_grouped.size()

203 ms ± 12.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit

not_valid_index = [
    1 
    for k, v in series_dict.items()
]

14.2 μs ± 347 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [19]:
_ = [v.index.freqstr for v in series_dict.values()]
type(_[0])

str

In [20]:
import pandas as pd

# Regular, con frecuencia
idx = pd.date_range('2024-01-01', periods=3, freq='D')
print(idx.freqstr)  # 'D'

# Irregular, sin frecuencia
idx2 = pd.DatetimeIndex(['2024-01-01', '2024-01-03', '2024-01-04'])
print(idx2.freq)      # None
print(idx2.freqstr)   # None

# Eliminar una fecha (pierde frecuencia)
idx3 = idx.delete(1)
print(idx3.freq)      # None
print(idx3.freqstr)   # None


D
None
None
None
None


In [22]:
pd.RangeIndex(start=0, stop=10, step=1, name='index', dtype='int64').freqstr

AttributeError: 'RangeIndex' object has no attribute 'freqstr'

In [23]:
exog = pd.DataFrame(
    {
        "exog_1": np.random.randint(1, 10, n),
        "exog_2": np.random.randint(1, 10, n),
        "exog_3": np.random.randint(1, 10, n),
        "exog_4": np.random.randint(1, 10, n),
        "exog_5": np.random.randint(1, 10, n),
        "exog_6": np.random.randint(1, 10, n),
    },
    index=pd.date_range(start="2023-01-01", periods=n, freq="D"),
)

In [27]:
not isinstance(exog.index, pd.MultiIndex)

True

In [28]:
exog.index.names

FrozenList([None])

In [26]:
not exog.index.names == ['series_id', 'datetime']

True